<a href="https://colab.research.google.com/github/qnhhoa/DecisionSupportSystem/blob/main/Passenger_Satisfaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Kết nối tới drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib.request
import seaborn as sns

%matplotlib inline

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Train set
DOWNLOAD_URL = 'https://raw.githubusercontent.com/qnhhoa/DecisionSupportSystem/main/train.csv'
DATASET_DIR_PATH = '/content/drive/MyDrive/Colab Notebooks/DSS' # [directory path] để save dataset trên google drive
DATASET_PATH = os.path.join(DATASET_DIR_PATH, 'passenger_satis_raw.csv')

: 

In [ ]:
# Test set
TEST_URL = 'https://raw.githubusercontent.com/qnhhoa/DecisionSupportSystem/main/test.csv'
TESTSET_PATH = os.path.join(DATASET_DIR_PATH, 'test.csv')

: 

# **Download and save dataset**

In [ ]:
def fetch_data(download_url, dataset_path):
    try:
        urllib.request.urlretrieve(download_url, dataset_path)
    except:
        print(f"failed to download and save dataset in {dataset_path}")

: 

In [ ]:
fetch_data(DOWNLOAD_URL, DATASET_PATH)
fetch_data(TEST_URL, TESTSET_PATH)

: 

# **Data Preprocessing**


In [ ]:
# load dataset
df = pd.read_csv(DATASET_PATH)

: 

In [ ]:
# load testset
test_set = pd.read_csv(TESTSET_PATH)

: 

In [ ]:
df1 = pd.concat([df, test_set])

: 

In [ ]:
# copy data ra một biến khác để đảm bảo an toàn cho dữ liệu
df_cp = df1.copy()

: 

## **EDA - Exploratory Data Analysis**

### Decribe data





In [ ]:
# kiểm tra 10 dòng đầu
df_cp.head(10)

: 

In [ ]:
# check info
df_cp.info()

: 

In [ ]:
df_cp.describe()

: 

In [ ]:
df_cp.drop(columns=['Unnamed: 0','id'],axis=1, inplace=True)

: 

In [ ]:
def bar_plot(variable):
    # we get the features
    var = df_cp[variable]
    # count number of categorical variable
    varValue = var.value_counts()
    
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n {}".format(variable,varValue))
     

: 

In [ ]:
categorical_var = ["Gender", "Customer Type", "Type of Travel", "Class","satisfaction"]
for each in categorical_var:
    bar_plot(each)

: 

In [ ]:
# check phân bố dữ liệu
df_cp.hist(bins=50,figsize=(40, 20))
plt.show()

: 

### Outliers

In [ ]:
# Lấy ra những cột dạng numeric trong data
numeric = df_cp.select_dtypes(exclude='object')
numeric

: 

In [ ]:
# Lấy ra những cột cần thiết
cols_to_check = [col for col in numeric.columns if col not in ['satisfaction']]
cols_to_check

: 

In [ ]:
# # visualize outliers sử dụng boxplot
# fig, ax = plt.subplots(6, 3, figsize = (25, 15))
# ax = ax.flatten()
# for i, c in enumerate(cols_to_check):
#     sns.boxplot(x = df[c], ax = ax[i])
# fig.tight_layout()

: 

### Handling Missing and Duplicate Data

In [ ]:
# check dữ liệu NaN
df_cp.isna().sum()

: 

In [ ]:
# xóa dữ liệu null
df_cp.dropna(inplace=True)

: 

In [ ]:
df_cp.duplicated().sum()

: 

In [ ]:
df_cp.head(10)

: 

## **Data Cleaning**

### Xử lý Outliers

In [ ]:
# outlier columns
outlier_cols = ["Flight Distance", "Checkin service"]

def outlier_thresholds(dataframe, variable):
    Q1 = df_cp[variable].quantile(0.25)
    Q3 = df_cp[variable].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR

    return lower_limit, upper_limit

def HandleOutliers(df):
    for col in outlier_cols:
        lower_limit, upper_limit = outlier_thresholds(df, col)
        print(f"{col} has {df[(df[col] < lower_limit) | (df[col] > upper_limit)].shape[0]} outliers")
        df = df[(df[col] >= lower_limit) & (df[col] <= upper_limit)]
    return df

: 

In [ ]:
df_cp = HandleOutliers(df_cp)

: 

In [ ]:
df_cp.reset_index(drop=True, inplace = True)

: 

In [ ]:
df_cp.head(10)

: 

## **Feature Engineering**

### Encode features

In [ ]:
df_cp["satisfaction"].value_counts()

: 

In [ ]:
df_cp["Gender"].value_counts()

: 

In [ ]:
df_cp["Class"].value_counts()

: 

In [ ]:
df_cp["Type of Travel"].value_counts()

: 

In [ ]:
df_cp["Customer Type"].value_counts()

: 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# encoder column
encoder = (OneHotEncoder(sparse=False,handle_unknown='ignore').fit_transform(df_cp[['Class']]))

# join to original df
# df_cp = df_cp.join(encoder)

encoder = pd.DataFrame(encoder)
encoder.columns = ['Business','Eco','Eco Plus']

: 

In [ ]:
encoder.head(10)

: 

In [ ]:
df_cp = df_cp.join(encoder)

: 

In [ ]:
# df_cp['Gender'] = df_cp["Gender"].map({value:idx for idx, value in enumerate(df_cp["Gender"].value_counts().index)})
# df_cp['Customer Type'] = df_cp["Customer Type"].map({value:idx for idx, value in enumerate(df_cp["Customer Type"].value_counts().index)})
# df_cp['Type of Travel'] = df_cp["Type of Travel"].map({value:idx for idx, value in enumerate(df_cp["Type of Travel"].value_counts().index)})
# df_cp['satisfaction'] = df_cp["satisfaction"].map({value:idx for idx, value in enumerate(df_cp["satisfaction"].value_counts().index)})

: 

In [ ]:
df_cp['Gender'].replace({'Male':1,'Female':0}, inplace=True)
df_cp['Type of Travel'].replace({'Personal Travel':1,'Business travel':0}, inplace=True)
df_cp['Customer Type'].replace({'Loyal Customer':1,'disloyal Customer':0}, inplace=True)
df_cp['satisfaction'].replace({'neutral or dissatisfied':0, 'satisfied':1}, inplace = True)

: 

In [ ]:
# results
df_cp.head(10)

: 

In [ ]:
df_cp.info()

: 

### Feature Selection

#### Chi-square test

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

: 

In [ ]:
categorical_features = df_cp[["Gender", "Customer Type",	"Age",	"Type of Travel",	"Inflight wifi service",	"Departure/Arrival time convenient",	"Ease of Online booking",
                              "Gate location", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", 
                              "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Eco Plus", "Eco", "Business"]]

: 

In [ ]:
best_features = SelectKBest(chi2, k=21)
best_features.fit(categorical_features, df_cp["satisfaction"])

: 

In [ ]:
df_score = pd.DataFrame(best_features.pvalues_,columns=['p_values'])
df_score['chi2_values'] = best_features.scores_
df_score['columns'] = ["Gender", "Customer Type",	"Age",	"Type of Travel",	"Inflight wifi service",	"Departure/Arrival time convenient",	"Ease of Online booking",
                              "Gate location", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", 
                              "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Eco Plus", "Eco", "Business"]
df_score.sort_values(by='p_values')

: 

In [ ]:
# chọn ra những features <= alpha
df_score[df_score['p_values'] <= 0.05]['columns']

: 

In [ ]:
# chọn ra những features > alpha
df_score[df_score['p_values'] > 0.05]['columns']

: 

#### ANOVA F-test

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

: 

In [ ]:
df_cp = df_cp.dropna()

: 

In [ ]:
continous_features  = df_cp[["Flight Distance","Departure Delay in Minutes", "Arrival Delay in Minutes"]]

: 

In [ ]:
best_features = SelectKBest(f_classif, k=3)
best_features.fit(continous_features , df_cp["satisfaction"])

: 

In [ ]:
df_score = pd.DataFrame(best_features.pvalues_,columns=['p_values'])
df_score['anova_values'] = best_features.scores_
df_score['columns'] = ["Flight Distance","Departure Delay in Minutes", "Arrival Delay in Minutes"]
df_score.sort_values(by='p_values')

: 

In [ ]:
# chọn ra những features <= alpha
df_score[df_score['p_values'] <= 0.05]['columns']

: 

### Data Scaling
Mục tiêu: Apply Data Normalization lên các continous features [Age, Flight Distance]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

: 

In [ ]:
cols_to_scale = ["Age", "Flight Distance"]
scaler = MinMaxScaler()
df_cp[cols_to_scale] = scaler.fit_transform(df_cp[cols_to_scale])

: 

In [ ]:
# result
df_cp[cols_to_scale]

: 

In [ ]:
df_cp.head(10)

: 

### Final result

In [ ]:
input = df_cp[["Flight Distance","Departure Delay in Minutes", "Arrival Delay in Minutes", "Gender", "Customer Type",	"Age",	"Type of Travel",	"Inflight wifi service",	"Departure/Arrival time convenient",	"Ease of Online booking",
                              "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", 
                              "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Eco Plus", "Eco", "Business"]].to_numpy()

: 

In [ ]:
input.shape

: 

In [ ]:
print(input)

: 

In [ ]:
y = df_cp["satisfaction"].to_numpy()

: 

In [ ]:
y.shape

: 

In [ ]:
print(y)

: 

# **Xây dựng mô hình**

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
import time

: 

In [ ]:
sns.set(font_scale=1.8)

: 

## Gaussian Naive Bayes

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, val_index = next(iter(sss.split(input, y)))
X_train, X_test = input[train_index], input[val_index]
y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
start = time.process_time()   

bayes = GaussianNB()
bayes.fit(X_train, y_train)
y_pred = bayes.predict(X_test)

totaltime = time.process_time() - start

: 

In [ ]:
print(f"time = {totaltime}")
print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
gauss_cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
plt.title("Confusion matrix with accuracy = {0}".format(accuracy_score(y_test, y_pred)), size=15)
ax = sns.heatmap(gauss_cm, annot=True, fmt="g", cmap="Blues", xticklabels=["satisfied", "neutral or dissatisfied"], yticklabels=["satisfied", "neutral or dissatisfied"])
ax.set_ylabel("True Label", fontsize=15)
ax.set_xlabel("Predict Label", fontsize=15)
plt.show()

: 

## Logistic Regression

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, val_index = next(iter(sss.split(input, y)))
X_train, X_test = input[train_index], input[val_index]
y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
start = time.process_time() 

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

totaltime = time.process_time() - start

: 

In [ ]:
print(f"time = {totaltime}")
print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
log_cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
plt.title("Confusion matrix with accuracy = {0}".format(accuracy_score(y_test, y_pred)), size=15)
ax = sns.heatmap(log_cm, annot=True, fmt="g", cmap="Blues", xticklabels=["satisfied", "neutral or dissatisfied"], yticklabels=["satisfied", "neutral or dissatisfied"])
ax.set_ylabel("True Label", fontsize=15)
ax.set_xlabel("Predict Label", fontsize=15)
plt.show()

: 

## Decision Trees

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, val_index = next(iter(sss.split(input, y)))
X_train, X_test = input[train_index], input[val_index]
y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
start = time.process_time()   

clf_gini = DecisionTreeClassifier()
clf_gini.fit(X_train, y_train)
y_pred = clf_gini.predict(X_test)

totaltime = time.process_time() - start

: 

In [ ]:
print(f"time = {totaltime}")
print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
dt_cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
plt.title("Confusion matrix with accuracy = {0}".format(accuracy_score(y_test, y_pred)), size=15)
ax = sns.heatmap(dt_cm, annot=True, fmt="g", cmap="Blues", xticklabels=["satisfied", "neutral or dissatisfied"], yticklabels=["satisfied", "neutral or dissatisfied"])
ax.set_ylabel("True Label", fontsize=15)
ax.set_xlabel("Predict Label", fontsize=15)
plt.show()

: 

## Random Forest

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, val_index = next(iter(sss.split(input, y)))
X_train, X_test = input[train_index], input[val_index]
y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
start = time.process_time()   

rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

totaltime = time.process_time() - start

: 

In [ ]:
print(f"time = {totaltime}")
print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
rf_cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
plt.title("Confusion matrix with accuracy = {0}".format(accuracy_score(y_test, y_pred)), size=15)
ax = sns.heatmap(rf_cm, annot=True, fmt="g", cmap="Blues", xticklabels=["satisfied", "neutral or dissatisfied"], yticklabels=["satisfied", "neutral or dissatisfied"])
ax.set_ylabel("True Label", fontsize=15)
ax.set_xlabel("Predict Label", fontsize=15)
plt.show()

: 

In [ ]:
# Saving model
import pickle as pkl
pkl.dump(rf, open("rf_model.pkl", "wb"))

: 

## XGBoost

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, val_index = next(iter(sss.split(input, y)))
X_train, X_test = input[train_index], input[val_index]
y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
start = time.process_time()   

xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)
y_pred = xgb_classifier.predict(X_test)

totaltime = time.process_time() - start

: 

In [ ]:
print(f"time = {totaltime}")
print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
xgb_cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 8))
plt.title("Confusion matrix with accuracy = {0}".format(accuracy_score(y_test, y_pred)), size=15)
ax = sns.heatmap(xgb_cm, annot=True, fmt="g", cmap="Blues", xticklabels=["satisfied", "neutral or dissatisfied"], yticklabels=["satisfied", "neutral or dissatisfied"])
ax.set_ylabel("True Label", fontsize=15)
ax.set_xlabel("Predict Label", fontsize=15)
plt.show()

: 

# GridSearchCV

In [ ]:
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import GridSearchCV
# from xgboost import XGBClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier

: 

In [ ]:
# X = df_cp[["Flight Distance","Departure Delay in Minutes", "Arrival Delay in Minutes", "Gender", "Customer Type",	"Age",	"Type of Travel",	"Inflight wifi service",	"Departure/Arrival time convenient",	"Ease of Online booking",
#                               "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", 
#                               "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Eco Plus", "Eco", "Business"]]
# y = df_cp["satisfaction"]

: 

In [ ]:
# smote = SMOTE(sampling_strategy='minority')
# X_sm, y_sm = smote.fit_resample(X, y)

: 

In [ ]:
# # check giá trị sau khi oversampling
# y_sm.value_counts()

: 

In [ ]:
# input = X_sm.to_numpy()
# y = y_sm.to_numpy()

: 

In [ ]:
# y.shape

: 

## Logistic Regression

In [ ]:
# from sklearn.model_selection import GridSearchCV
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# train_index, val_index = next(iter(sss.split(input, y)))
# X_train, X_test = input[train_index], input[val_index]
# y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
# logreg = LogisticRegression()
# parameters = [{'C':[0.1,0.5, 1.0, 10.0], 
#                'solver':['sag', 'liblinear', 'lbfgs']}]

# model_lr_grid = GridSearchCV(estimator = logreg,  
#                            param_grid = parameters,
#                            scoring = 'accuracy',
#                            cv = 3,
#                            verbose=0)

: 

In [ ]:
# model_lr_grid.fit(X_train, y_train)

: 

In [ ]:
# model_lr_grid.best_params_

: 

In [ ]:
# y_pred = model_lr_grid.predict(X_test)

: 

In [ ]:
# print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
# print(classification_report(y_test, y_pred, target_names=["leave", "stay"]))

: 

## XGBOOST

In [ ]:
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1715)

# train_index, val_index = next(iter(sss.split(input, y)))
# X_train, X_test = input[train_index], input[val_index]
# y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
# model_xgb= XGBClassifier()
# parameters = {'n_estimators':[10, 20, 30, 40, 50, 100, 200],'max_depth':[3, 4, 5, 6],'learning_rate':[0.1, 0.5, 1]}
# model_xgb_grid = GridSearchCV(model_xgb, 
#                               parameters,
#                               cv=3,
#                               verbose=0)

: 

In [ ]:
# model_xgb_grid.fit(X_train, y_train)

: 

In [ ]:
# model_xgb_grid.best_params_

: 

In [ ]:
# y_pred = model_xgb_grid.predict(X_test)

: 

In [ ]:
# print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
# print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

In [ ]:
# load the model from disk
loaded_model = pickle.load(open("rf_model.pkl", 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

: 

In [ ]:
X_test.shape

: 

## Random Forest

In [ ]:
# from sklearn.model_selection import GridSearchCV
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# train_index, val_index = next(iter(sss.split(input, y)))
# X_train, X_test = input[train_index], input[val_index]
# y_train, y_test = y[train_index], y[val_index]

: 

In [ ]:
# rfc=RandomForestClassifier(random_state=42)
# param_grid = { 
#     'n_estimators': [200, 500],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,5,6,7,8],
#     'criterion' :['gini', 'entropy']
# }

: 

In [ ]:
# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)

: 

In [ ]:
# CV_rfc.fit(X_train, y_train)

: 

In [ ]:
# CV_rfc.best_params_

: 

In [ ]:
# y_pred = CV_rfc.predict(X_test)

: 

In [ ]:
# print(f"Accuracy = {accuracy_score(y_test, y_pred)}")
# print(classification_report(y_test, y_pred, target_names=["satisfied", "neutral or dissatisfied"]))

: 

# **Application**

In [ ]:
!pip install streamlit
!pip install pyngrok===4.1.1
from pyngrok import ngrok

: 

In [ ]:
%%writefile healthy-heart-app.py
import streamlit as st
import base64
import sklearn
import numpy as np
import pickle as pkl
from sklearn.preprocessing import MinMaxScaler
scal=MinMaxScaler()

#Load the saved model
model = pickle.load(open("rf_model.pkl", "rb"))

st.set_page_config(page_title="Healthy Heart App",page_icon="⚕️",layout="centered",initial_sidebar_state="expanded")

def preprocess(distance ,depDelay ,arrDelay ,gender ,cusType ,age ,typetrvl ,wifi ,timeConve ,booking ,food ,board ,seat ,entertain ,onboard ,leg ,bag ,checkin ,inflight ,clean ,cusClass ):   

     if gender == "Male":
         gender = 1
     else:
         gender = 0
     if cusType == "disloyal Customer":
         cusType = 0
     else:
         cusType = 1
     if typetrvl == "Business travel":
         typetrvl = 0
     else:
         typetrvl = 1
    
     li_class = { "business": 0, "eco": 0, "eco plus": 0}
     if items.cusClass in li_class:
         li_class[items.cusClass] = 1

     age = (items.age - df["Age"].min()) / (df["Age"].max() - df["Age"].min())
     distance = (items.distance - df["Flight Distance"].min()) / (df["Flight Distance"].max() - df["Flight Distance"].min())

    user_input=[distance ,depDelay ,arrDelay ,gender ,cusType ,age ,typetrvl ,wifi ,timeConve ,booking ,food ,board ,seat ,entertain ,onboard ,leg ,bag ,checkin ,inflight ,clean ,cusClass ]
    user_input=np.array(user_input)
    user_input=user_input.reshape(1,-1)
    user_input=scal.fit_transform(user_input)
    prediction = model.predict(user_input)

    return prediction

    # front end elements of the web page 
html_temp = """ 
    <div style ="background-color:pink;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Healthy Heart App</h1> 
    </div> 
    """
      
# display the front end aspect
st.markdown(html_temp, unsafe_allow_html = True) 
st.subheader('by Amlan Mohanty ')
      
# following lines create boxes in which user can enter data required to make prediction
distance=st.number_input('Insert flight distance')
depDelay = st.number_input('Insert Departure Delay in Minutes')
arrDelay = st.number_input('Insert Arrival Delay in Minutes')
gender = st.radio("Select Gender: ", ('Male', 'Female'))
cusType = st.radio("Customer Type?",("Loyal Customer", "disloyal Customer"))
age=st.selectbox ("Age",range(1,121,1))
typetrvl = st.radio("Type of travel?",("Business travel", "Personal Travel"))
wifi = st.select_slider("Inflight wifi service?",options=["1","2", "3", "4", "5"])
timeConve = st.select_slider("Departure/Arrival time convenient?",options=["1","2", "3", "4", "5"])
booking = st.select_slider("Ease of Online booking?",options=["1","2", "3", "4", "5"])
food = st.select_slider("Food and drink?",options=["1","2", "3", "4", "5"])
board = st.select_slider("Online boarding?",options=["1","2", "3", "4", "5"])
seat = st.select_slider("Seat comfort?",options=["1","2", "3", "4", "5"])
entertain= st.select_slider("Inflight entertainment?",options=["1","2", "3", "4", "5"])
onboard = st.select_slider("On-board service?",options=["1","2", "3", "4", "5"])
leg = st.select_slider("Leg room service?",options=["1","2", "3", "4", "5"])
bag = st.select_slider("Baggage handling?",options=["1","2", "3", "4", "5"])
checkin = st.select_slider("Checkin service?",options=["1","2", "3", "4", "5"])
inflight = st.select_slider("Inflight service?",options=["1","2", "3", "4", "5"])
clean = st.select_slider("Cleanlines?",options=["1","2", "3", "4", "5"])
class = st.selectbox('Customer Class?',("Business", "Eco", "Eco Plus")) 


pred=preprocess(distance ,depDelay ,arrDelay ,gender ,cusType ,age ,typetrvl ,wifi ,timeConve ,booking ,food ,board ,seat ,entertain ,onboard ,leg ,bag ,checkin ,inflight ,clean ,cusClass )


if st.button("Predict"):    
  if pred[0] == 0:
    st.error('neutral or dissatisfied')
    
  else:
    st.success('satisfied')


st.sidebar.info("This web app is helps you to find out whether you are at a risk of developing a heart disease.")
st.sidebar.info("Enter the required fields and click on the 'Predict' button to check whether you have a healthy heart")



: 

In [ ]:
!nohup streamlit run healthy-heart-app.py &
url = ngrok.connect(port='8501')
url

: 

In [ ]:
# import requests

# import streamlit as st

# # Define the title
# # st.title("Car evaluation web application")
# # st.write(
# #     "The model evaluates a cars acceptability based on the inputs below.\
# #     Pass the appropriate details about your car using the questions below to discover if your car is acceptable."
# # )

# # Input 1
# gender = st.radio(
#     "Gender?",
#     ("Female","Male")
# )
# # Input 2
# cusType = st.radio(
#     "Customer Type?",
#     ("Loyal Customer", "disloyal Customer")
# )
# # Input 3
# age = st.number_input('Insert your age')
# # Input 4
# typetrvl = st.radio(
#     "Type of travel?",
#     ("Business travel", "Personal Travel")
# )
# #Input 5
# cusClass = st.radio(
#     "Class?",
#     ("Business", "Eco", "Eco Plus")
# )
# #Input 6
# distance = st.number_input('Insert flight distance')
# #Input 7
# depDelay = st.number_input('Insert Departure Delay in Minutes')
# #Input 8
# arrDelay = st.number_input('Insert Arrival Delay in Minutes')
# #Input 9
# wifi = st.select_slider(
#     "Inflight wifi service?",
#     options=["1","2", "3", "4", "5"]
# )
# # Input 10
# timeConve = st.select_slider(
#     "Departure/Arrival time convenient?",
#     options=["1","2", "3", "4", "5"]
# )
# # Input 11
# booking = st.select_slider(
#     "Ease of Online booking?",
#     options=["1","2", "3", "4", "5"]
# )
# # Input 12
# food = st.select_slider(
#     "Food and drink?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 13
# board = st.select_slider(
#     "Online boarding?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 14
# seat = st.select_slider(
#     "Seat comfort?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 15
# entertain= st.select_slider(
#     "Inflight entertainment?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 16
# onboard = st.select_slider(
#     "On-board service?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 17
# leg = st.select_slider(
#     "Leg room service?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 18
# bag = st.select_slider(
#     "Baggage handling?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 19
# checkin = st.select_slider(
#     "Checkin service?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 20
# inflight = st.select_slider(
#     "Inflight service?",
#     options=["1","2", "3", "4", "5"]
# )
# #Input 21
# clean = st.select_slider(
#     "Cleanlines?",
#     options=["1","2", "3", "4", "5"]
# )
# # Class values to be returned by the model
# class_values = {
#     0: "neutral or dissatisfied",
#     1: "satisfied"
#     }
# # When 'Submit' is selected
# if st.button("Submit"):

#     # Inputs to ML model
#     inputs = {
#         "inputs": [
#             {
#                 "gender": gender,
#                 "cusType": cusType,
#                 "age": age,
#                 "typetrvl": typetrvl,
#                 "cusClass": cusClass,
#                 "distance": distance,
#                 "depDelay": depDelay,
#                 "arrDelay": arrDelay,
#                 "wifi": wifi,
#                 "timeConve": timeConve,
#                 "booking": booking,
#                 "food": food,
#                 "board": board,
#                 "seat": seat,
#                 "entertain": entertain,
#                 "onboard": onboard,
#                 "leg": leg,
#                 "bag": bag,
#                 "checkin": onboard,
#                 "inflight": inflight,
#                 "clean": clean
#             }
#         ]
#         }
       
#     # Posting inputs to ML API
#     response = requests.post(f"http://host.docker.internal:8001/api/v1/predict/", json=inputs, verify=False)
#     json_response = response.json()

#     prediction = class_values[json_response.get("predictions")[0]]

#     st.subheader(f"This car is **{prediction}!**")

: 

In [ ]:
# !pip install pydantic
# !pip install fastapi
# !pip install uvicorn
# !pip install colabcode

: 

In [ ]:
# from pydantic import BaseModel
# from fastapi import FastAPI
# from colabcode import ColabCode

: 

In [ ]:
# cc = ColabCode(port=5000, code=False)

: 

In [ ]:
# df_cp.info()

: 

In [ ]:
# class ITEMS(BaseModel):
#     distance: float
#     depDelay: int
#     arrDelay: float
#     gender: str
#     cusType: str
#     age: int
#     typetrvl: str
#     wifi: int
#     timeConve: int
#     booking: int
#     food: int
#     board: int
#     seat: int
#     entertain: int
#     onboard: int
#     leg: int
#     bag: int
#     checkin: int
#     inflight: int
#     clean: int
#     cusClass: str



: 

In [ ]:
# app = FastAPI()
# model = pickle.load(open("rf_model.pkl", "rb"))

# @app.get("/")
# async def index():
#     return {"text": "hello"}

# @app.put("/predict")
# async def predict(items: ITEMS):
#     if items.gender == "Male":
#         gender = 1
#     else:
#         gender = 0
#     if items.cusType == "disloyal Customer":
#         cusType = 0
#     else:
#         cusType = 1
#     if items.typetrvl == "Business travel":
#         typetrvl = 0
#     else:
#         typetrvl = 1
    
#     li_class = { "business": 0, "eco": 0, "eco plus": 0}
#     if items.cusClass in li_class:
#         li_class[items.cusClass] = 1

#     age = (items.age - df["Age"].min()) / (df["Age"].max() - df["Age"].min())
#     distance = (items.distance - df["Flight Distance"].min()) / (df["Flight Distance"].max() - df["Flight Distance"].min())

    # types = {
    #    0: "neutral or dissatisfied 0",
    #    1: "satisfied 1"
    # }
   
#     prediction = list(map(lambda x: types[x], model.predict([[distance, items.depDelay,items.arrDelay, gender,cusType, age, typetrvl, items.wifi,items.timeConve, items.booking, items.food, items.board, items.seat, items.entertain, items.onboard, items.leg, items.bag, items.checkin, items.inflight, items.clean,li_class["eco"], li_class["eco plus"], li_class["business"]]]).tolist()))
#     return {"predict": prediction}

: 

In [ ]:
# !ngrok authtoken 29Jiw4zse9fyhQTqost16CJgRHc_33rG2WxCMejiooaWj3mHB

: 

In [ ]:
# cc.run_app(app=app)

: 